# 🧾 LIDL pirkimų istorijos analizė

Šis notebook’as leidžia paprastai ir greitai peržiūrėti **Jūsų** „Lidl“ pirkimų statistiką *(jeigu pirkdami naudojatės LIDL programėle)*. Projektas skirtas **mokymuisi ir asmeniniam naudojimui**.

## 🔑 Prisijungimas prie „Lidl“ duomenų

Kad šis kodas galėtų gauti Jūsų pirkinių informaciją, jam reikia **Jūsų prisijungimo rakto (auth token)**.
👉 Be šio rakto kodas **neveiks**.

> ⚠️ **SVARBU**  
> - Rakto negalima dalintis  
> - Jis **laikinas ir asmeninis**  
> - Baigus darbą, rekomenduojama **ištrinti raktą**

---

## Kaip įrašyti savo raktą?

1. Prisijunkite prie savo „Lidl“ paskyros naršyklėje
2. Paspauskite **F12** (atsidarys kūrėjo įrankiai)
3. Raskite Cookies/Slapukai savo naršyklėje:
    - 🦊 **Mozilla Firefox**: Storage/Saugykla → Cookies/Slapukai  
    - 🌐 **Google Chrome**: Application/Programa → Storage/Saugykla → Cookies/Slapukai
4. Pasirinkite svetainę `https://www.lidl...`
5. Nukopijuokite **authToken** slapuko reikšmę
6. Įklijuokite ją čia:
```python
auth_token = "JŪSŲ_AUTH_TOKEN"
```
7. Šiame Google Colab notebook’e paspauskite **Run all/ Vykdyti viską** ⬆️
8. Palaukite **30–60 sekundžių**, kol bus atlikta analizė

In [ ]:
# @title
print("Renkami duomenys, palaukite...")

import requests
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt

base_url = "https://www.lidl.lt/mre/api/v1/tickets"
country = "LT"

headers = {
    "Cookie": f"authToken={auth_token}"
}


all_tickets = []
page = 1

while True:
    response = requests.get(base_url, headers=headers, params={"country": country, "page": page})

    if response.status_code != 200:
        print(f"Error: status code {response.status_code} on page {page}")
        break

    try:
        data = response.json()
    except Exception as e:
        print(f"Failed to decode JSON on page {page}: {e}")
        break

    all_tickets.extend(
    {
        "id": item["id"],
        "date": item["date"],
        "totalAmount": item["totalAmount"],
        "articlesCount": item["articlesCount"],
        "store": item["store"],
    }
    for item in data["items"]
    )

    if page * data.get("size", 0) >= data.get("totalCount", 0):
        break
    page += 1

    time.sleep(0.2)

# Save
df = pd.DataFrame(all_tickets)

df["id"] = df["id"].astype("string")
df["date"] = pd.to_datetime(df["date"], utc=True)
df["totalAmount"] = pd.to_numeric(df["totalAmount"], errors="coerce")
df["articlesCount"] = pd.to_numeric(df["articlesCount"], errors="coerce").astype("Int64")
df["store"] = df["store"].astype("string")

print(f"Surinkti {len(df)} kvitai.")


In [ ]:
# @title
total_spent = df["totalAmount"].sum()
print(f"\U0001f4b0 Iš viso išleista: {total_spent:.2f} Eur")

avg_basket = df["articlesCount"].mean()
print(f"\n\U0001f4a1 Vidutiniškai pirkinių krepšelis sudarytas iš: {avg_basket:.1f} prekių vienetų.")

avg_spent = df["totalAmount"].mean()
print(f"\U0001f4a1 Vidutinė pirkinių krepšelio suma: {avg_spent:.2f} Eur.")

In [ ]:
# @title
store_spend = df.groupby("store")["totalAmount"].sum().sort_values(ascending=False)

sns.set(style="whitegrid")
plt.figure(figsize=(8,5))
sns.barplot(x=store_spend.index, y=store_spend.values, hue = store_spend.index, palette="muted")

plt.legend([],[], frameon=False)
plt.title("Išlaidos pagal parduotuves")
plt.xlabel("Parduotuvės")
plt.ylabel("Išlaidos, Eur")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# @title
year_spend = df.groupby(df["date"].dt.year)["totalAmount"].sum()
year_spend = year_spend.sort_index()

sns.set(style="whitegrid")
plt.figure(figsize=(8,5))
sns.barplot(x=year_spend.index, y=year_spend.values, hue = year_spend.index, palette="muted")

plt.legend([],[], frameon=False)
plt.title("Išlaidos pagal metus")
plt.xlabel("Metai")
plt.ylabel("Išlaidos, Eur")
plt.show()

In [ ]:
# @title
month_spend_2025 = df[df["date"].dt.year == 2025].groupby(df["date"].dt.month)["totalAmount"].sum()
month_spend_2025 = month_spend_2025.sort_index()

sns.set(style="whitegrid")
plt.figure(figsize=(8,5))
sns.barplot(x=month_spend_2025.index, y=month_spend_2025.values, hue = month_spend_2025.index, palette="muted")

plt.legend([],[], frameon=False)
plt.title("2025 metų išlaidos pagal mėnesius")
plt.xlabel("Mėnesiai")
plt.ylabel("Išlaidos, Eur")
plt.show()